In [15]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

# Change to MPS logic if on mac
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
# Put the data in your Google Drive
# You ca get the data here: https://www.kaggle.com/competitions/titanic/data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [18]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [19]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/content/drive/MyDrive/Data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [20]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

# Only use pairs where the data has the prefix above
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [21]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['c est une tres gentille fille .', 'she s a very nice girl .']


In [22]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)

    def forward(self, input, hidden_cell):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden_cell = self.lstm(output, hidden_cell)
        return output, hidden_cell

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [23]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden_cell):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden_cell = self.lstm(output, hidden_cell)
        output = self.softmax(self.out(output[0]))
        return output, hidden_cell

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [24]:
# Split a sentence by ' ' and return a list of the tokens (int ids) for each word
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

# Call the above on a sentence
# After calling, add the EOS_token (int id) to the gotten list
def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

# For a source, target pair, call the above. Return a tuple of 2 tensors, one input_tensor and another an output_tensor
def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [25]:
# For 50% of pairs, use teacher forcing so that we predict (y_1, y_2, ..., y_{T}) from (y_0, y_1, ..., y_{T-1}) on the decoder side
# Without teacher forcing, we start with y_0 = SOS_token and then use \hat{y}_1, the prediction at time step 0 as the input to time step 1 on the decoder side
# For this case, we'll predict (\hat{y}_1, \hat{y}_2, ..., \hat{y}_{T}) from (y_0, \hat{y}_1, ..., \hat{y}_{T-1}) on the decoder side
# The crucial thing to realize here is that \hat{y}_t is stochastic, and dependent on what the model predicts - mistakes propegate!
teacher_forcing_ratio = 0.5

def train(
    input_tensor,
    target_tensor,
    encoder,
    decoder,
    encoder_optimizer,
    decoder_optimizer,
    criterion,
    max_length=MAX_LENGTH
):
    encoder_hidden = encoder.initHidden()
    encoder_cell = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    # Flip the data
    # If we want to predict [x, y, z] from [a, b, c], we should feed in [c, b, a] on the encoder side so that a is as close to x as possible
    input_tensor = input_tensor.flip(0)

    # The above trick was noted in making a big difference
    for it in range(input_length):
        encoder_output, (encoder_hidden, encoder_cell) = encoder(
            input_tensor[it], (encoder_hidden, encoder_cell)
        )

    decoder_input = torch.tensor([[SOS_token]], device=device)

    # Initialize the hidden states of the decoder with the hidden states of the encoder
    decoder_hidden = encoder_hidden
    decoder_cell = encoder_cell

    # For this pair, use teacher forcing with 50% probability, else don't
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    target_length_used = 0

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input

        target_length_used = target_length

        for jt in range(target_length):
            decoder_output, (decoder_hidden, decoder_cell) = decoder(
                decoder_input, (decoder_hidden, decoder_cell)
            )
            loss += criterion(decoder_output, target_tensor[jt])
            decoder_input = target_tensor[jt]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        # Note that in this case we feed in at most target_length words
        # If, however, we predict EOS_token, we break out
        # You need to also carefully get the target lenght used since it might not be target_length
        for jt in range(target_length):
            decoder_output, (decoder_hidden, decoder_cell) = decoder(
                decoder_input, (decoder_hidden, decoder_cell)
            )
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # Detach from history as input

            loss += criterion(decoder_output, target_tensor[jt])

            target_length_used += 1

            if decoder_input.item() == EOS_token:
                break

    # Collect gradients
    loss.backward()

    # Do a step, both for the encoder and the decoder
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length_used

In [26]:
# Helper functions used to time the optimizations
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [27]:
def trainIters(
    encoder,
    decoder,
    n_iters,
    print_every=1000,
    plot_every=100,
    learning_rate=0.01
):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    # Initialize the encoder and decoder optimizers with the above learning rate
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    # Get n_iters training pairs
    # In this example, we are effectively doing SGD with batch size 1
    training_pairs = [
        tensorsFromPair(random.choice(pairs))
        for i in range(n_iters)
    ]

    # The loss; either NLLLoss if you use log sigmoids or CrossEntropyLoss if you use logits
    criterion = nn.NLLLoss()

    for it in range(1, n_iters + 1):
        training_pair = training_pairs[it - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        # Train on the input, target pair
        loss = train(
            input_tensor, target_tensor, encoder,
            decoder, encoder_optimizer, decoder_optimizer, criterion
        )

        # Update the total loss and the plot loss
        # We can plot and print at different granularities
        print_loss_total += loss
        plot_loss_total += loss

        if it % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, it / n_iters),
                                         it, it / n_iters * 100, print_loss_avg))

        if it % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [28]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [29]:
# For a certain input, get the predicted output sentence
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_cell = encoder.initHidden()

        for it in range(input_length):
            encoder_output, (encoder_hidden, encoder_cell) = encoder(
                input_tensor[it],
                (encoder_hidden, encoder_cell)
            )

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden
        decoder_cell = encoder_cell

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for jt in range(max_length):
            decoder_output, (decoder_hidden, decoder_cell) = decoder(
                decoder_input,
                (decoder_hidden, decoder_cell)
            )
            topv, topi = decoder_output.data.topk(1)

            decoded_words.append(output_lang.index2word[topi.item()])

            if topi.item() == EOS_token:
                break

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [30]:
hidden_size = 256
cell_size = 256
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder, decoder, 75000, print_every=5000)

7m 23s (- 103m 32s) (5000 6%) 3.1246
14m 51s (- 96m 31s) (10000 13%) 2.4720


KeyboardInterrupt: ignored

Here we will do an evaluation.
Gather up n=7500 random pairs and for each pair get te BLEU score.
For this exercise, don't use BLUE based on 4-grams, use just 2-grams.
Investigate how this can be done with "sentence_bleu" in nltk - you can also find other implementations.
Print the average BLEU score after you've randomly drawn the sentences. You should be a BLEU of about 10 or slightly more. Note that is is training BLEU, which is all I'd like you to get.

In [31]:
from nltk.translate.bleu_score import sentence_bleu

def evaluateRandomly(encoder, decoder, n=7500, debug=False):
    bleu_scores = []
    for i in range(n):
        pair = random.choice(pairs)
        if debug:
            print('French Original: ', pair[0])
            print('English Reference: ', pair[1])
        # Leave out the <EOS> symbol
        output_words = evaluate(encoder, decoder, pair[0])

        if '<EOS>' == output_words[-1]:
            output_words.pop()

        output_sentence = ' '.join(output_words)
        score = 100.0 * sentence_bleu([pair[1].split(' ')], output_words, weights=[0.5, 0.5])
        bleu_scores.append(score)
        if debug:
            print('Candidate Translation: ', output_sentence)
            print('BLEU: ', score)
            print('')
    print('The mean BLEU score is: ', np.mean(bleu_scores))

In [32]:
evaluateRandomly(encoder, decoder)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


The mean BLEU score is:  18.80714355176119
